# ⚡ Otimização de Arquiteturas Multi-Agent com DSPy

**Versão:** 1.0 - Multi-Agent Optimization  
**Nível:** Avançado  
**Tempo estimado:** 60-90 minutos  
**Abordagem:** Teoria + Prática + Experimentos

---

## 📋 Sobre este Notebook

Este notebook explora **técnicas avançadas de otimização** para sistemas multi-agent com DSPy.

### 🎯 O Desafio

Otimizar sistemas multi-agent é **significativamente mais complexo** que otimizar agentes individuais:

❌ **Por que?**
- Múltiplos agentes = múltiplas signatures para otimizar
- Interdependências entre agentes
- Espaço de busca exponencialmente maior
- Trade-offs entre agentes (melhorar um pode piorar outro)
- Métricas de qualidade mais complexas

✅ **O que você vai aprender:**
- Como adaptar técnicas de otimização para multi-agent
- MIPRO para arquiteturas cognitivas
- Técnicas customizadas por arquitetura
- Joint optimization vs Independent optimization
- Métricas e avaliação para sistemas complexos
- Estratégias de debugging e refinamento

### 📚 Pré-requisitos

**IMPORTANTE:** Complete antes:
1. [Multi-Agent & Arquiteturas Cognitivas](dspy_multiagent_cognitive_architectures.ipynb)
2. [Otimização Avançada](dspy_agents_advanced_linear_final.ipynb) ou [Hands-On](dspy_agents_advanced_handson_final.ipynb)

### 🗺️ Roadmap

1. **Fundamentos de Otimização Multi-Agent**
   - Desafios únicos
   - Estratégias de otimização
   - Joint vs Independent

2. **Otimização por Arquitetura**
   - Hierarchical: Coordenador + Especialistas
   - Sequential: Pipeline end-to-end
   - Collaborative: Debate e consenso
   - Reflexive: Loop de refinamento

3. **Técnicas Avançadas**
   - MIPRO adaptado
   - Meta-prompting para coordenação
   - Reward shaping
   - Quality-aware optimization

4. **Implementação Prática**
   - Datasets para cada arquitetura
   - Métricas customizadas
   - Experimentos comparativos

---

**Let's optimize! 🚀**


## 📚 Parte 1: Fundamentos de Otimização Multi-Agent

### O que muda ao otimizar Multi-Agent?

#### Agente Individual vs Multi-Agent

| Aspecto | Agente Individual | Multi-Agent |
|---------|-------------------|-------------|
| **Signatures** | 1 signature | N signatures (N agentes) |
| **Exemplos** | Simples: input → output | Complexos: múltiplas interações |
| **Métricas** | Direta: accuracy, F1 | Compostas: qualidade + coordenação |
| **Espaço de Busca** | Linear | Exponencial (N!) |
| **Dependências** | Nenhuma | Forte acoplamento |

### 🎯 Estratégias de Otimização

#### 1. **Independent Optimization (Independente)**
```
Otimizar cada agente separadamente
├── Vantagens: Simples, paralelizável, rápido
└── Desvantagens: Ignora interdependências, subótimo
```

#### 2. **Sequential Optimization (Sequencial)**
```
Otimizar agentes em ordem
├── Vantagens: Captura algumas dependências
└── Desvantagens: Ordem importa, ainda subótimo
```

#### 3. **Joint Optimization (Conjunta)**
```
Otimizar todos juntos simultaneamente
├── Vantagens: Captura todas interdependências, ótimo global
└── Desvantagens: Complexo, lento, custoso
```

#### 4. **Iterative Optimization (Iterativa)**
```
Ciclos de otimização independente + joint
├── Vantagens: Balance entre velocidade e qualidade
└── Desvantagens: Requer múltiplas iterações
```

### 🔑 Conceitos Chave

#### Meta-Prompting
Usar prompts que coordenam outros agentes:
```python
# Coordenador aprende COMO coordenar
"Given agent outputs: {A, B, C}, synthesize best result"
```

#### Reward Shaping
Definir recompensas intermediárias:
```python
# Não apenas resultado final
total_reward = 0.3 * search_quality + 0.5 * recommendation_quality + 0.2 * coordination
```

#### Quality Metrics Composition
Compor métricas de múltiplos agentes:
```python
# Métrica composta
def multi_agent_metric(prediction, gold):
    agent_scores = [evaluate_agent_i(prediction, gold) for i in agents]
    coordination_score = evaluate_coordination(prediction)
    return weighted_average(agent_scores + [coordination_score])
```

### 📊 Desafios Específicos

#### 1. **Explosion Combinatória**
- N agentes com M exemplos cada = M^N combinações
- Solução: Sampling inteligente, pruning

#### 2. **Credit Assignment**
- Qual agente causou o erro?
- Solução: Métricas por agente + composição

#### 3. **Overfitting Coordenado**
- Agentes "conspiram" nos exemplos de treino
- Solução: Validação cruzada entre agentes

#### 4. **Trade-offs entre Agentes**
- Melhorar A pode piorar B
- Solução: Multi-objective optimization


## Setup e Imports


In [1]:
import dspy
import os
from datetime import datetime
from typing import List, Optional, Dict, Any, Tuple, Callable
from pydantic import BaseModel, Field
from dataclasses import dataclass
import json
import uuid
from enum import Enum
from dotenv import load_dotenv
import numpy as np
from collections import defaultdict

load_dotenv()


True

In [ ]:
# Configurar LLM
lm = dspy.LM('openai/gpt-4o-mini')
dspy.configure(lm=lm)


## Data Models e Database (Reutilizados)


In [ ]:
# NOTA: Copie os data models e database do notebook anterior
# dspy_multiagent_cognitive_architectures.ipynb
# Incluindo: UserProfile, Flight, Itinerary, users_db, flights_db, ferramentas

# Por brevidade, vamos importar direto (assuma que executou notebook anterior)
# Ou copie as células relevantes aqui

# Models básicos
class UserProfile(BaseModel):
    name: str
    user_id: str
    email: str
    phone: str
    frequent_flyer_number: Optional[str] = None
    preferences: Dict[str, Any] = Field(default_factory=dict)

class Flight(BaseModel):
    flight_id: str
    flight_number: str
    departure_airport: str
    arrival_airport: str
    departure_time: str
    arrival_time: str
    duration_minutes: int
    price: float
    available_seats: int
    airline: str = "Default Airlines"

# Simplified database for demonstration
users_db = {
    "Adam": UserProfile(
        name="Adam",
        user_id="user_001",
        email="adam@example.com",
        phone="+1-555-0101",
        preferences={"priority": "price"}
    ),
    "Sarah": UserProfile(
        name="Sarah",
        user_id="user_002",
        email="sarah@example.com",
        phone="+1-555-0102",
        preferences={"priority": "duration"}
    )
}

flights_db = {
    "SFO-JFK": [
        Flight(flight_id="f001", flight_number="AA101", departure_airport="SFO",
               arrival_airport="JFK", departure_time="08:00", arrival_time="16:30",
               duration_minutes=330, price=450.00, available_seats=15, airline="American Airlines"),
        Flight(flight_id="f002", flight_number="UA205", departure_airport="SFO",
               arrival_airport="JFK", departure_time="14:00", arrival_time="22:45",
               duration_minutes=345, price=380.00, available_seats=8, airline="United"),
        Flight(flight_id="f003", flight_number="DL150", departure_airport="SFO",
               arrival_airport="JFK", departure_time="11:00", arrival_time="19:20",
               duration_minutes=320, price=520.00, available_seats=12, airline="Delta"),
    ]
}


## 🎯 Parte 2: Otimização da Arquitetura Hierarchical

### Desafios Específicos

A arquitetura **Hierarchical** tem um coordenador e múltiplos especialistas:

```
      [Coordinator]
      /    |     \
  [S1]  [S2]  [S3]
```

**Desafios:**
1. **Coordenador** precisa aprender a rotear corretamente
2. **Especialistas** precisam ser experts em suas áreas
3. **Interdependência**: Coordenador depende da qualidade dos especialistas

### 🔧 Estratégias de Otimização

#### 1. **Bottom-Up Optimization**
Otimizar especialistas primeiro, depois coordenador

```python
# Passo 1: Otimizar cada especialista independentemente
optimized_specialist_1 = optimize(specialist_1, specialist_1_data)
optimized_specialist_2 = optimize(specialist_2, specialist_2_data)

# Passo 2: Otimizar coordenador com especialistas fixos
optimized_coordinator = optimize(coordinator, full_data, 
                                  fixed_specialists=[s1, s2])
```

**Vantagens:**
- ✅ Especialistas se tornam muito bons em suas áreas
- ✅ Paralelizável (especialistas independentes)
- ✅ Mais rápido

**Desvantagens:**
- ⚠️ Coordenador pode não saber usar especialistas otimizados
- ⚠️ Subótimo global

#### 2. **Top-Down Optimization**
Otimizar coordenador primeiro, depois especialistas

```python
# Passo 1: Otimizar coordenador com especialistas baseline
optimized_coordinator = optimize(coordinator, routing_data)

# Passo 2: Otimizar especialistas baseado em roteamento aprendido
for specialist in specialists:
    routed_data = get_routed_data(optimized_coordinator, specialist)
    optimized_specialist = optimize(specialist, routed_data)
```

**Vantagens:**
- ✅ Especialistas otimizados para casos reais
- ✅ Coordenação alinhada com especialização

**Desvantagens:**
- ⚠️ Especialistas baseline podem ser ruins
- ⚠️ Sequencial (mais lento)

#### 3. **Alternating Optimization (Recomendado)**
Alternar otimização entre coordenador e especialistas

```python
for iteration in range(max_iterations):
    # Otimizar especialistas com coordenador fixo
    for specialist in specialists:
        specialist = optimize(specialist, specialist_data)
    
    # Otimizar coordenador com especialistas atualizados
    coordinator = optimize(coordinator, routing_data, 
                          specialists=specialists)
    
    # Avaliar melhoria
    if improvement < threshold:
        break
```

**Vantagens:**
- ✅ Captura interdependências
- ✅ Converge para melhor solução
- ✅ Balance entre qualidade e tempo

### 💡 Técnicas Específicas

#### Meta-Prompting para Coordenador
Ensinar o coordenador a COMO usar especialistas:

```python
meta_examples = [
    "When user asks about SEARCH → use SearchSpecialist",
    "When user asks about RECOMMENDATION → use RecommendationSpecialist",
    "When user mentions BOOKING → use BookingSpecialist"
]
```

#### Specialist-Specific Metrics
Métricas diferentes para cada especialista:

```python
def evaluate_search(pred, gold):
    return recall_at_k(pred.flights, gold.flights, k=3)

def evaluate_recommendation(pred, gold):
    return precision(pred.top_flight, gold.best_flight)

def evaluate_coordinator(pred, gold):
    return routing_accuracy(pred.specialist_choice, gold.correct_specialist)
```

### 🎯 Implementação


In [ ]:
# Exemplo: Hierarchical Alternating Optimization

class HierarchicalOptimizer:
    """
    Otimizador para arquitetura hierarchical usando alternating optimization
    """
    
    def __init__(self, coordinator, specialists: Dict[str, dspy.Module]):
        self.coordinator = coordinator
        self.specialists = specialists
        self.history = []
        
    def optimize_specialists(self, trainset, metric_fn):
        """Otimizar cada especialista independentemente"""
        print("🔧 Optimizing Specialists...")
        
        optimized_specialists = {}
        for name, specialist in self.specialists.items():
            print(f"  → Optimizing {name}...")
            
            # Filtrar exemplos relevantes para este especialista
            specialist_data = [ex for ex in trainset if ex.specialist_type == name]
            
            if len(specialist_data) > 0:
                # Usar BootstrapFewShot para especialista
                optimizer = dspy.BootstrapFewShot(
                    metric=metric_fn,
                    max_bootstrapped_demos=4,
                    max_labeled_demos=4
                )
                
                optimized = optimizer.compile(
                    specialist,
                    trainset=specialist_data
                )
                optimized_specialists[name] = optimized
                print(f"    ✅ {name} optimized with {len(specialist_data)} examples")
            else:
                optimized_specialists[name] = specialist
                print(f"    ⚠️ {name} skipped (no data)")
        
        return optimized_specialists
    
    def optimize_coordinator(self, trainset, specialists, metric_fn):
        """Otimizar coordenador com especialistas fixos"""
        print("\n🎯 Optimizing Coordinator...")
        
        # Métrica específica para coordenador (routing accuracy)
        def coordinator_metric(example, pred, trace=None):
            # Verifica se escolheu o especialista correto
            correct = (pred.required_specialist == example.correct_specialist)
            return float(correct)
        
        optimizer = dspy.BootstrapFewShot(
            metric=coordinator_metric,
            max_bootstrapped_demos=8,
            max_labeled_demos=8
        )
        
        optimized_coordinator = optimizer.compile(
            self.coordinator,
            trainset=trainset
        )
        
        print("  ✅ Coordinator optimized")
        return optimized_coordinator
    
    def alternating_optimize(self, trainset, metric_fn, max_iterations=3):
        """
        Alternating optimization: especialistas → coordenador → especialistas → ...
        """
        print("=" * 60)
        print("🔄 ALTERNATING OPTIMIZATION")
        print("=" * 60)
        
        current_specialists = self.specialists.copy()
        current_coordinator = self.coordinator
        
        for iteration in range(max_iterations):
            print(f"\n🔄 ITERATION {iteration + 1}/{max_iterations}")
            print("-" * 60)
            
            # Passo 1: Otimizar especialistas
            current_specialists = self.optimize_specialists(
                trainset, 
                metric_fn
            )
            
            # Passo 2: Otimizar coordenador
            current_coordinator = self.optimize_coordinator(
                trainset,
                current_specialists,
                metric_fn
            )
            
            # Avaliar sistema completo
            score = self.evaluate_system(
                current_coordinator,
                current_specialists,
                trainset,
                metric_fn
            )
            
            self.history.append({
                'iteration': iteration + 1,
                'score': score,
                'coordinator': current_coordinator,
                'specialists': current_specialists.copy()
            })
            
            print(f"\n📊 System Score after iteration {iteration + 1}: {score:.3f}")
            
            # Early stopping se não houver melhoria
            if iteration > 0:
                prev_score = self.history[iteration - 1]['score']
                improvement = score - prev_score
                print(f"   Improvement: {improvement:+.3f}")
                
                if improvement < 0.01:  # threshold
                    print("   → Converged! Stopping early.")
                    break
        
        return current_coordinator, current_specialists
    
    def evaluate_system(self, coordinator, specialists, testset, metric_fn):
        """Avaliar sistema hierarchical completo"""
        total_score = 0
        for example in testset:
            try:
                # Simular execução do sistema
                # (Simplified - em produção seria mais complexo)
                total_score += 1  # Placeholder
            except:
                pass
        
        return total_score / max(len(testset), 1)


## Parte 3: Otimização Sequential Pipeline

Arquitetura: A1 → A2 → A3 → Output

**Estratégia: Backward Optimization (de trás para frente)**
- Otimizar último agente primeiro (próximo ao output)
- Depois penúltimo, e assim por diante
- Cada agente aprende o que o próximo espera


In [ ]:
# Sequential/Pipeline Optimizer
class SequentialPipelineOptimizer:
    """Backward optimization para pipelines"""
    
    def __init__(self, agents: List[dspy.Module]):
        self.agents = agents  # [A1, A2, A3, A4] em ordem
        
    def backward_optimize(self, trainset, metrics):
        """Otimizar de trás para frente"""
        print("🔄 BACKWARD OPTIMIZATION")
        print("=" * 60)
        
        optimized_agents = [None] * len(self.agents)
        
        # Otimizar de trás para frente
        for i in range(len(self.agents) - 1, -1, -1):
            print(f"\n📍 Optimizing Agent {i+1}/{len(self.agents)} (position {i})")
            
            agent = self.agents[i]
            metric = metrics.get(f'agent_{i}', metrics['default'])
            
            # Criar dados para este agente
            agent_trainset = self._prepare_agent_data(trainset, i, optimized_agents)
            
            # Otimizar
            optimizer = dspy.BootstrapFewShot(
                metric=metric,
                max_bootstrapped_demos=4
            )
            
            optimized = optimizer.compile(agent, trainset=agent_trainset)
            optimized_agents[i] = optimized
            
            print(f"  ✅ Agent {i} optimized")
        
        return optimized_agents
    
    def _prepare_agent_data(self, trainset, agent_idx, optimized_agents):
        """Preparar dados específicos para cada agente no pipeline"""
        # Se temos agentes otimizados depois, usar suas entradas esperadas
        # Implementação simplificada
        return trainset
    
    def end_to_end_optimize(self, trainset, metric):
        """Otimização end-to-end do pipeline completo"""
        print("\n🎯 END-TO-END OPTIMIZATION")
        
        class FullPipeline(dspy.Module):
            def __init__(self, agents):
                super().__init__()
                self.agents = agents
            
            def forward(self, **kwargs):
                x = kwargs
                for agent in self.agents:
                    x = agent(**x)
                return x
        
        pipeline = FullPipeline(self.agents)
        
        optimizer = dspy.MIPROv2(
            metric=metric,
            num_candidates=5,
            init_temperature=1.0
        )
        
        optimized = optimizer.compile(
            pipeline,
            trainset=trainset,
            num_trials=10
        )
        
        print("  ✅ Pipeline optimized end-to-end")
        return optimized


## 📚 Conteúdo Completo e Próximos Passos

### Conteúdo Detalhado

Este notebook foi complementado com um guia executivo completo em:
**`MULTIAGENT_OPTIMIZATION_SUMMARY.md`**

O documento inclui:
- ✅ Otimização detalhada de TODAS as 4 arquiteturas
- ✅ MIPRO adaptado para cada arquitetura
- ✅ Datasets e métricas customizadas
- ✅ Experimentos comparativos
- ✅ Best practices e recomendações
- ✅ Código executável para cada técnica

### Resumo das Técnicas

| Arquitetura | Técnica Principal | MIPRO Config | Complexidade |
|-------------|------------------|--------------|--------------|
| **Hierarchical** | Alternating Optimization | 10 candidates, 30 trials | ⭐⭐⭐ |
| **Sequential** | Backward + End-to-End | 8 candidates, 20 trials | ⭐⭐⭐⭐ |
| **Collaborative** | Reward Shaping + Multi-Objective | 12 candidates, 40 trials | ⭐⭐⭐⭐⭐ |
| **Reflexive** | Actor-Critic Co-Optimization | 8 candidates, 25 trials | ⭐⭐⭐⭐ |

### 🚀 Próximos Passos

1. **Implemente sua arquitetura**
2. **Crie datasets customizados** para seu domínio
3. **Defina métricas apropriadas** (compostas!)
4. **Comece com otimização simples** (BootstrapFewShot)
5. **Evolua para MIPRO** quando tiver dados suficientes
6. **Itere e refine** baseado em resultados

### 🎯 Recomendações Finais

- **Start Simple**: BootstrapFewShot → MIPRO
- **Measure Everything**: Custos, latência, qualidade por agente
- **Custom Metrics**: Invista tempo nisso!
- **Iterate**: Não otimize tudo de uma vez
- **Document**: Trace o que funciona e o que não funciona

---

**Série Completa de Notebooks DSPy:**
1. Fundamentos Básicos (Linear)
2. Fundamentos Básicos (Hands-On)  
3. Otimização Avançada (Linear)
4. Otimização Avançada (Hands-On)
5. Multi-Agent & Arquiteturas Cognitivas
6. **→ Otimização Multi-Agent** (você está aqui!)

Happy Optimizing! ⚡
